In [1]:
from newspaper import Article
from collectors.wikipedia import *
from collectors.google_news import *
from collectors.trends import *

import pandas as pd


/opt/anaconda3/envs/rag_llamaindex/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0
/opt/anaconda3/e

### Get Trending celebrities

In [2]:
trendCollector = TrendCollector()
trending_celebs = await trendCollector.get_trending_celeb(limit=100)
trending_celebs

['John Goodman',
 'Jennifer Aniston',
 'Eva Longoria',
 'Sydney Sweeney',
 'Justin Timberlake',
 'Evan Rachel Wood',
 'Don Jr',
 'Anita De Monte',
 'Blake Lively',
 'Justin Baldoni',
 'Bradley Cooper',
 'James Gunn',
 'Marlon Brando',
 'J. D. Vance',
 'Rihanna',
 'Riot',
 'Jeremy Renner',
 'Craig Conover',
 'Joe D ’ Amelio',
 'Paige DeSorbo',
 'Stevie Wonder',
 'Jeff Bezos',
 'Jessica Alba',
 '##fer',
 'Bill Skarsgard',
 'Pat McAfee',
 'Camille Gottlieb',
 'Grace Kelly',
 'Olivia Cooke',
 'Robin Wright',
 'Elizabeth Hurley',
 'Aaron Phypers',
 'Denise Richards',
 'Jessica Biel',
 'Klay Thompson',
 'Megan Thee Stallion',
 'Will Smith',
 'Gregg Wallace',
 'Roseanne Barr',
 'Juliana Aidén Martinez',
 'Kourtney Kardashian',
 'Penelope',
 'Baekhyun',
 'Stellan Skarsgard',
 'Kanye “ Ye',
 'West',
 'Bruce Willis',
 'Rumi Carter',
 'Adam Levine',
 'James Righton',
 'Keira Knightley',
 'Morgan Wallen',
 'Mary - Louise Parker',
 'Anandhi',
 'Pre',
 'Priyadarshi',
 'Barbie',
 'Harper Beckham',
 '

### Get wiki and profile infos via Wikipedia

In [3]:
wikiCollector = WikipediaCollector()
profiles = []

for name in trending_celebs:
    wikiCollector.save_wiki(name)
    profile = wikiCollector.profile(name)
    profiles.append(profile)

07/15/2025 03:41:45 AM - Failed to save wiki of John Goodman: (Page id "john good man" does not match any pages. Try another id!)
en.wikipedia.org (parse) John Goodman
en.wikipedia.org (imageinfo) File:John Goodman by Gage Skidmore.jpg
John Goodman (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:John Good...
  infobox: <dict(12)> name, image, caption, birth_name, birth_date...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:J...
  pageid: 268009
  parsetree: <str(50881)> <root><template><title>Short description...
  requests: <list(2)> parse, imageinfo
  title: John Goodman
  wikibase: Q215072
  wikidata_url: https://www.wikidata.org/wiki/Q215072
  wikitext: <str(42209)> {{Short description|American actor (born ...
}
07/15/2025 03:41:46 AM - Successfully saved wiki of Jennifer Aniston
en.wikipedia.org (parse) Jennifer Aniston
en.wikipedia.org (imageinfo) File:JenniferAnistonHWoFFeb2012.jpg
Jennifer Aniston (en) data
{
  image: <list(1)> {'kind': 'p

In [7]:
from dataclasses import asdict

profile_dicts = [asdict(p) for p in profiles]
celeb_profiles = pd.DataFrame(profile_dicts)
celeb_profiles

TypeError: asdict() should be called on dataclass instances

In [15]:
news_collector = CelebrityGoogleNewsCollector()
# celebrity_news = await news_collector.search_celebrities_news(max_results=10)
# celebrity_news

celebrity_news = await news_collector.search(search_query="Justin Bieber AND Marriage", max_results=10)
celebrity_news

[GoogleNews(title='Justin Bieber admits to Hailey Bieber marriage struggles in new album - Fox News', description='Justin Bieber admits to Hailey Bieber marriage struggles in new album  Fox News', published_date='Fri, 11 Jul 2025 15:32:00 GMT', url='https://www.foxnews.com/entertainment/justin-bieber-admits-hailey-bieber-marriage-struggles-new-album', publisher='Fox News'),
 GoogleNews(title='Justin Bieber’s Latest Vacation Post With Hailey Quietly Addresses Marriage Trouble Rumors - ELLE', description='Justin Bieber’s Latest Vacation Post With Hailey Quietly Addresses Marriage Trouble Rumors  ELLE', published_date='Wed, 09 Jul 2025 17:54:00 GMT', url='https://www.elle.com/culture/celebrities/a65350597/justin-bieber-hailey-spain-pda-photos-marriage-rumors-response/', publisher='ELLE'),
 GoogleNews(title='Justin Bieber Sends Subtle Message About Hailey Bieber Marriage - Us Weekly', description='Justin Bieber Sends Subtle Message About Hailey Bieber Marriage  Us Weekly', published_date='

In [16]:
from collectors.utility.text_processing import *

articles = []

for i in range(len(celebrity_news)):
    try:
        news = celebrity_news[i]
        article = await news_collector.get_full_article(news.url)
        article.nlp()
        
        ppl, event_type, event = extract_event(article.summary)
        
        articles.append({
            'title' : article.title,
            'description' : news.description,
            'celebrities' : ppl,
            'event_type' : event_type,
            'event' : event,
            'keywords' : article.keywords,
            'text' : article.text,
            'url' : article.url or article.source_url
        })
    except Exception as e:
        continue

An error occurred while fetching the article: Article `download()` failed with 404 Client Error: Not Found for url: https://www.foxnews.com/entertainment/justin-bieber-admits-hailey-bieber-marriage-struggles-new-album on URL https://www.foxnews.com/entertainment/justin-bieber-admits-hailey-bieber-marriage-struggles-new-album


In [18]:
df = pd.DataFrame(articles)
df

,title,description,celebrities,event_type,event,keywords,text,url
0,Justin Bieber’s New Post With Hailey Addresses...,Justin Bieber’s Latest Vacation Post With Hail...,"[##Justin Bieber, Bie, Hailey, Hailey Bieber, ...",EventCategory.CAREER,Event.COMEBACK,"[reports, hailey, addresses, bieber, wife, jus...",Every item on this page was chosen by an ELLE ...,https://www.elle.com/culture/celebrities/a6535...
1,Justin Bieber Sends Subtle Message About Haile...,Justin Bieber Sends Subtle Message About Haile...,"[Baldwin, Belie, Hailey, Hailey Bieber, Justin...",EventCategory.RELATIONSHIP,Event.DATING,"[selena, life, bieber, hailey, going, justin, ...",Justin Bieber offered insight into his dynamic...,https://www.usmagazine.com/celebrity-news/news...
2,Justin Bieber Reveals Hailey Bieber Relationsh...,Justin Bieber Reveals Hailey Bieber Relationsh...,"[##ustin Bieber, Hailey, Hailey Bieber, Justin...",EventCategory.SOCIAL_MEDIA,Event.STATEMENT,"[n, amid, status, bieber, hailey, wife, photos...",Watch : Justin Bieber Calls Hailey Bieber His ...,https://www.eonline.com/news/1419499/justin-bi...
3,Justin Bieber Addresses Rumors of Marriage Str...,Justin Bieber Addresses Rumors of Marriage Str...,"[Bieber, Justin Bieber]",EventCategory.OTHERS,Event.OTHER,"[life, bieber, album, biebers, wife, justin, s...",Justin Bieber is still a wife guy on his new a...,https://www.rollingstone.com/music/music-news/...
4,"What Justin Bieber Says About Marriage, Divorc...","What Justin Bieber Says About Marriage, Divorc...","[Hailey Bieber, Justin Bieber]",EventCategory.SOCIAL_MEDIA,Event.STATEMENT,"[music, divorce, bieber, hailey, wife, justin,...","After a four-year music hiatus, Justin Bieber ...",https://www.today.com/popculture/music/justin-...
5,‘Defenseless’ Justin Bieber confirms Hailey ma...,‘Defenseless’ Justin Bieber confirms Hailey ma...,"[Hailey, Hailey Bieber, Justin, Justin Bieber]",EventCategory.PERSONAL,Event.FAMILY,"[defenseless, hailey, bieber, album, lilbieber...",Justin Bieber is hitting rumors head-on.\n\nTh...,https://pagesix.com/2025/07/11/entertainment/j...
6,Justin Bieber Says He Won't Be 'Walking Away' ...,Justin Bieber Says He Won't Be 'Walking Away' ...,"[##in, B, Hailey Bieber, Justin, Justin Bieber]",EventCategory.SOCIAL_MEDIA,Event.STATEMENT,"[sings, hailey, wife, bieber, aint, justin, sh...",NEED TO KNOW Justin Bieber released his sevent...,https://people.com/justin-bieber-not-leave-wif...
7,The Biebers Would Like Us to Know They’re Stil...,The Biebers Would Like Us to Know They’re Stil...,"[Hailey Bieber, Justin, Justin Bieber]",EventCategory.OTHERS,Event.OTHER,"[sings, cover, hailey, bieber, justin, busines...",The happy couple. Photo: XNY/Star Max/GC Image...,https://www.vulture.com/article/justin-bieber-...
8,"Justin Bieber addresses marriage woes, mental ...","Justin Bieber addresses marriage woes, mental ...","[Bieber, Hailey, Justin Bieber]",EventCategory.SOCIAL_MEDIA,Event.STATEMENT,"[im, music, bieber, album, hailey, million, ju...",Justin Bieber has released one of his most per...,https://nypost.com/2025/07/11/entertainment/ju...


In [5]:
df.iloc[3][['title', 'celebrities', 'event', 'keywords', 'url']]

title          Donovan Mitchell, Coco Jones celebrate engagem...
celebrities    [Coco Jones, Donovan Mitchell, Jones, Russell ...
event                                            Event.STATEMENT
keywords       [russell, pair, life, grammy, coco, engagement...
url            https://nypost.com/2025/07/12/sports/donovan-m...
Name: 3, dtype: object

In [6]:
df[['title', 'event', 'url']].to_csv("sample.csv")